Я буду рассматривать только игроков, родившихся после 1930ого года.

Вопросы и гипотезы:
1) Из какой страны было взято наибольшее число игроков?
2) Какое значение параметра "Points per Game Played" у игроков входящих в 5% лучших? (считаем, что игроки с большим P/GP лучше, чем с меньшим)
3) Есть ли связь между ростом игрока и среднем количеством забитых им голов за игру? (предполагаю, что нет)
4) Правда ли, что люди с большим количеством забитых победных голов реже помогают забивать?
5) Можно ли выразить количество забитых голов с помощью линейной функции от количества забитых победных голов?
6) Правда ли, что средний рост игроков, родившихся до 1980ого года меньше, чем тех, кто родился позже?

In [2]:
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt

# Подготовка датасета

Для начала объединим таблицы Bio info (0-78) и Summury (0-78), а также удалим одинаковые строки. В итоге должно остаться 7735 строк

In [3]:
def get_bio_info_table():
    list_of_bio_info = []
    for i in range(79):
        list_of_bio_info.append(pd.read_excel("nhl/bio/Bio info (" + str(i) + ").xlsx"))
    bio_info = pd.concat(list_of_bio_info, axis=0)
    bio_info = bio_info.drop_duplicates()
    return bio_info
bio_info = get_bio_info_table()
bio_info

,Player,S/C,Pos,DOB,Birth City,S/P,Ctry,Ntnlty,Ht,Wt,Draft Yr,Round,Overall,1st Season,HOF,GP,G,A,P
0,Jack Laviolette,R,D,1879-07-27,Belleville,ON,CAN,CAN,71,170,--,--,--,19171918,Y,18,2,1,3
1,Ken Thompson,L,L,1881-05-29,Oakengates,--,GBR,GBR,70,160,--,--,--,19171918,N,1,0,0,0
2,Joe Hall,R,D,1882-05-03,Staffordshire,--,GBR,GBR,70,175,--,--,--,19171918,Y,37,15,9,24
3,Didier Pitre,R,R,1883-09-01,Valleyfield,QC,CAN,CAN,71,185,--,--,--,19171918,Y,127,64,35,99
4,Lester Patrick,L,D,1883-12-30,Drummondville,QC,CAN,CAN,73,180,--,--,--,19261927,Y,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30,Bradly Nadeau,R,L,2005-05-05,St-Francois de Madaw,NB,CAN,CAN,71,172,2023,1,30,20232024,N,1,0,0,0
31,Zach Benson,L,L,2005-05-12,Chilliwack,BC,CAN,CAN,70,170,2023,1,13,20232024,N,112,19,27,46
32,Connor Bedard,R,C,2005-07-17,North Vancouver,BC,CAN,CAN,70,185,2023,1,1,20232024,N,115,35,67,102
33,Macklin Celebrini,L,C,2006-06-13,North Vancouver,BC,CAN,CAN,72,190,2024,1,1,20242025,N,38,14,20,34


In [4]:
def get_summary_table():
    list_of_summary = []
    for i in range(79):
        list_of_summary.append(pd.read_excel("nhl/summary/Summary (" + str(i) + ").xlsx"))
    summary = pd.concat(list_of_summary, axis=0)
    summary = summary.drop_duplicates()
    return summary
summary = get_summary_table()
summary

,Player,S/C,Pos,GP,G,A,P,+/-,PIM,P/GP,...,PPG,PPP,SHG,SHP,OTG,GWG,S,S%,TOI/GP,FOW%
0,Antti Aalto,L,C,151,11,17,28,-25,52,0.19,...,3,4,0,0,0,1,182,6,11:03,49.7
1,Reg Abbott,L,C,3,0,0,0,0,0,0.00,...,0,0,0,0,0,0,--,--,--,--
2,Spencer Abbott,R,L,2,0,0,0,-2,0,0.00,...,0,0,0,0,0,0,3,0,6:55,--
3,Justin Abdelkader,L,L,739,106,146,252,-56,608,0.34,...,25,48,1,3,3,15,"1,238",8.6,14:54,50.5
4,Clarence Abel,L,D,333,20,18,38,0,365,0.11,...,0,0,1,1,1,5,--,--,--,--
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30,Jason Zucker,L,L,743,212,193,405,-3,386,0.55,...,43,88,3,5,3,32,"1,683",12.6,15:30,32
31,Mike Zuke,R,C,455,86,196,282,-42,220,0.62,...,21,105,3,7,0,6,754,11.4,--,--
32,Rudy Zunich,L,D,2,0,0,0,0,2,0.00,...,0,0,0,0,0,0,--,--,--,--
33,Valentin Zykov,R,L,55,7,10,17,-2,10,0.31,...,1,3,0,0,0,1,69,10.1,11:07,18.2


In [5]:
print(bio_info.columns)
print(summary.columns)

Index(['Player', 'S/C', 'Pos', 'DOB', 'Birth City', 'S/P', 'Ctry', 'Ntnlty',
       'Ht', 'Wt', 'Draft Yr', 'Round', 'Overall', '1st Season', 'HOF', 'GP',
       'G', 'A', 'P'],
      dtype='object')
Index(['Player', 'S/C', 'Pos', 'GP', 'G', 'A', 'P', '+/-', 'PIM', 'P/GP',
       'EVG', 'EVP', 'PPG', 'PPP', 'SHG', 'SHP', 'OTG', 'GWG', 'S', 'S%',
       'TOI/GP', 'FOW%'],
      dtype='object')


Теперь объединим таблицы в одну

In [6]:
player_info = pd.merge(left=bio_info, right=summary, left_on=['Player', 'S/C', 'Pos', 'GP'], right_on=['Player', 'S/C', 'Pos', 'GP'])
player_info

,Player,S/C,Pos,DOB,Birth City,S/P,Ctry,Ntnlty,Ht,Wt,...,PPG,PPP,SHG,SHP,OTG,GWG,S,S%,TOI/GP,FOW%
0,Jack Laviolette,R,D,1879-07-27,Belleville,ON,CAN,CAN,71,170,...,--,--,--,--,0,0,--,--,--,--
1,Ken Thompson,L,L,1881-05-29,Oakengates,--,GBR,GBR,70,160,...,--,--,--,--,0,0,--,--,--,--
2,Joe Hall,R,D,1882-05-03,Staffordshire,--,GBR,GBR,70,175,...,--,--,--,--,0,1,--,--,--,--
3,Didier Pitre,R,R,1883-09-01,Valleyfield,QC,CAN,CAN,71,185,...,--,--,--,--,0,12,--,--,--,--
4,Lester Patrick,L,D,1883-12-30,Drummondville,QC,CAN,CAN,73,180,...,--,--,--,--,0,0,--,--,--,--
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7730,Bradly Nadeau,R,L,2005-05-05,St-Francois de Madaw,NB,CAN,CAN,71,172,...,0,0,0,0,0,0,3,0,19:08,--
7731,Zach Benson,L,L,2005-05-12,Chilliwack,BC,CAN,CAN,70,170,...,1,4,0,0,0,1,159,12,14:41,21.2
7732,Connor Bedard,R,C,2005-07-17,North Vancouver,BC,CAN,CAN,70,185,...,9,39,0,0,1,5,318,11,19:50,36.2
7733,Macklin Celebrini,L,C,2006-06-13,North Vancouver,BC,CAN,CAN,72,190,...,4,10,0,0,1,3,125,11.2,19:35,48.7


In [7]:
print(player_info.columns)

Index(['Player', 'S/C', 'Pos', 'DOB', 'Birth City', 'S/P', 'Ctry', 'Ntnlty',
       'Ht', 'Wt', 'Draft Yr', 'Round', 'Overall', '1st Season', 'HOF', 'GP',
       'G_x', 'A_x', 'P_x', 'G_y', 'A_y', 'P_y', '+/-', 'PIM', 'P/GP', 'EVG',
       'EVP', 'PPG', 'PPP', 'SHG', 'SHP', 'OTG', 'GWG', 'S', 'S%', 'TOI/GP',
       'FOW%'],
      dtype='object')


In [8]:
player_info['DOB'] = pd.to_datetime(player_info['DOB'], format='%Y-%m-%d')

In [9]:
player_info = player_info[player_info['DOB'] >= pd.Timestamp('1930-01-01')]

In [10]:
player_info.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6736 entries, 999 to 7734
Data columns (total 37 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   Player      6736 non-null   object        
 1   S/C         6736 non-null   object        
 2   Pos         6736 non-null   object        
 3   DOB         6736 non-null   datetime64[ns]
 4   Birth City  6736 non-null   object        
 5   S/P         6736 non-null   object        
 6   Ctry        6736 non-null   object        
 7   Ntnlty      6736 non-null   object        
 8   Ht          6736 non-null   int64         
 9   Wt          6736 non-null   object        
 10  Draft Yr    6736 non-null   object        
 11  Round       6736 non-null   object        
 12  Overall     6736 non-null   object        
 13  1st Season  6736 non-null   int64         
 14  HOF         6736 non-null   object        
 15  GP          6736 non-null   object        
 16  G_x         6736 non-null  

In [11]:
player_info.describe()

,DOB,Ht,1st Season,G_x,G_y,+/-,P/GP,OTG,GWG
count,6736,6736.000000,6.736000e+03,6736.000000,6736.000000,6736.000000,6736.000000,6736.000000,6736.000000
mean,1973-12-19 07:49:14.394299312,72.524941,1.995594e+07,52.166716,52.166716,-2.460214,0.299678,0.664935,7.409293
min,1930-01-05 00:00:00,0.000000,1.948195e+07,0.000000,0.000000,-257.000000,0.000000,0.000000,0.000000
25%,1960-07-30 06:00:00,71.000000,1.981198e+07,1.000000,1.000000,-18.250000,0.140000,0.000000,0.000000
50%,1975-04-02 12:00:00,73.000000,1.997200e+07,11.000000,11.000000,-3.000000,0.250000,0.000000,1.000000
75%,1989-04-29 12:00:00,74.000000,2.011201e+07,57.000000,57.000000,1.000000,0.430000,0.000000,8.000000
max,2006-08-21 00:00:00,81.000000,2.024202e+07,894.000000,894.000000,722.000000,1.920000,27.000000,135.000000
std,NaN,3.074618,1.805848e+05,94.885942,94.885942,53.861465,0.237624,1.920033,14.168603


In [12]:
player_info.sample(10)

,Player,S/C,Pos,DOB,Birth City,S/P,Ctry,Ntnlty,Ht,Wt,...,PPG,PPP,SHG,SHP,OTG,GWG,S,S%,TOI/GP,FOW%
2537,Brad McCrimmon,L,D,1959-03-29,Dodsland,SK,CAN,CAN,71,197,...,15,77,8,29,0,15,"1,557",5.2,--,--
3064,Gord Kluzak,L,D,1964-03-04,Climax,SK,CAN,CAN,76,220,...,8,49,1,2,0,3,467,5.4,--,--
6926,Blake Hillman,L,D,1996-01-26,Elk River,MN,USA,USA,73,193,...,0,0,1,1,0,0,5,20,18:02,--
1167,Gord Redahl,L,R,1935-08-28,Kinistino,SK,CAN,CAN,71,170,...,0,0,0,0,0,0,--,--,--,--
7177,Fredrik Karlstrom,L,C,1998-01-12,Stockholm,--,SWE,SWE,75,195,...,0,0,0,0,0,0,7,0,8:18,--
4837,Joe DiPenta,R,D,1979-02-25,Barrie,ON,CAN,CAN,74,220,...,0,0,0,0,0,1,67,9,12:35,0
4699,Mike York,R,L,1978-01-03,Waterford,MI,USA,USA,70,185,...,32,95,7,14,1,23,"1,206",10.5,17:09,46.7
2083,John Paddock,--,R,1954-06-09,Brandon,MB,CAN,CAN,75,187,...,0,0,0,0,0,0,83,9.6,--,--
6126,Conor Allen,L,D,1990-01-31,Chicago,IL,USA,USA,73,210,...,0,0,0,0,0,0,4,0,13:10,--
4323,Eric Landry,L,C,1975-01-20,Gatineau,QC,CAN,CAN,71,185,...,2,3,0,0,0,0,62,8.1,9:20,51.9


#

In [13]:
print(player_info.groupby('Ctry')['Ctry'].agg('count'))

Ctry
AUS       1
AUT      10
BEL       2
BHR       1
BHS       1
BLR      13
BRA       1
CAN    4048
CHE      36
CZE     237
DEU      36
DNK      16
EST       1
FIN     222
FRA      13
GBR      14
HRV       2
HTI       1
IDN       1
ITA       4
JAM       1
JPN       1
KAZ       5
KOR       2
LBN       1
LTU       3
LVA      22
NGA       2
NLD       3
NOR       9
POL       6
PRY       1
RUS     274
SRB       1
SVK      91
SVN       2
SWE     378
TWN       1
TZA       1
UKR      17
USA    1252
UZB       1
VEN       2
Name: Ctry, dtype: int64
